In [2]:
import os
import zipfile as zf
import pandas as pd
import numpy as np
import duckdb
pd.options.display.max_colwidth = 15
pd.options.display.max_rows = 4

In [3]:
files_names = sorted(os.listdir('./data/processed/parquet/'))
files_names

['dfp_cia_aberta_2010.parquet',
 'dfp_cia_aberta_2011.parquet',
 'dfp_cia_aberta_2012.parquet',
 'dfp_cia_aberta_2013.parquet',
 'dfp_cia_aberta_2014.parquet',
 'dfp_cia_aberta_2015.parquet',
 'dfp_cia_aberta_2016.parquet',
 'dfp_cia_aberta_2017.parquet',
 'dfp_cia_aberta_2018.parquet',
 'dfp_cia_aberta_2019.parquet',
 'dfp_cia_aberta_2020.parquet',
 'dfp_cia_aberta_2021.parquet',
 'itr_cia_aberta_2020.parquet',
 'itr_cia_aberta_2021.parquet',
 'itr_cia_aberta_2022.parquet']

In [5]:
sql = """
SELECT *
FROM parquet_scan('data/processed/parquet/*.parquet')
WHERE CD_CVM = 9512;
"""
df = duckdb.query(sql).to_df()
df

,CD_CVM,CNPJ_CIA,DENOM_CIA,GRUPO_DFP,VERSAO,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,2,2020-12-31,NaT,2019-12-31,-1,1,Ativo Total,True,NaN,9.260110e+11
1,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,2,2020-12-31,NaT,2019-12-31,-1,1.01,Ativo Circu...,True,NaN,1.121010e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37112,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2013-12-31,2013-01-01,2013-12-31,0,3.99.02.01,ON,False,NaN,1.790000e+03
37113,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2013-12-31,2013-01-01,2013-12-31,0,3.99.02.02,PN,False,NaN,1.790000e+03


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37114 entries, 0 to 37113
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   CD_CVM         37114 non-null  int32         
 1   CNPJ_CIA       37114 non-null  object        
 2   DENOM_CIA      37114 non-null  object        
 3   GRUPO_DFP      37114 non-null  object        
 4   VERSAO         37114 non-null  int8          
 5   DT_REFER       37114 non-null  datetime64[ns]
 6   DT_INI_EXERC   25026 non-null  datetime64[ns]
 7   DT_FIM_EXERC   37114 non-null  datetime64[ns]
 8   ORDEM_EXERC    37114 non-null  int8          
 9   CD_CONTA       37114 non-null  object        
 10  DS_CONTA       37114 non-null  object        
 11  ST_CONTA_FIXA  37114 non-null  bool          
 12  COLUNA_DF      13916 non-null  object        
 13  VL_CONTA       37114 non-null  float64       
dtypes: bool(1), datetime64[ns](3), float64(1), int32(1), int8(2), object(6

In [7]:
df[df.COLUNA_DF.notnull()]

,CD_CVM,CNPJ_CIA,DENOM_CIA,GRUPO_DFP,VERSAO,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
348,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,2,2020-12-31,2019-01-01,2019-12-31,-1,5.01,Saldos Inic...,True,Capital Soc...,2.054320e+11
349,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,2,2020-12-31,2019-01-01,2019-12-31,-1,5.01,Saldos Inic...,True,Reservas de...,2.674000e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36848,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2013-12-31,2013-01-01,2013-12-31,0,5.07,Saldos Finais,True,Outros Resu...,-7.243606e+09
36849,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2013-12-31,2013-01-01,2013-12-31,0,5.07,Saldos Finais,True,Patrimônio ...,3.481398e+11


In [11]:
# to use a database file (not shared between processes)
con = duckdb.connect(database='./data/db/my-db.duckdb', read_only=False)
con

In [12]:
sql = "CREATE TABLE teste_tb AS SELECT * FROM parquet_scan('data/processed/parquet/*.parquet');"
con.execute(sql).fetchall()

[(17608711,)]

In [13]:
df.GRUPO_DFP.unique()
# df

array(['DF Consolidado - Balanço Patrimonial Ativo',
       'DF Consolidado - Balanço Patrimonial Passivo',
       'DF Consolidado - Demonstração das Mutações do Patrimônio Líquido',
       'DF Consolidado - Demonstração de Resultado Abrangente',
       'DF Consolidado - Demonstração de Valor Adicionado',
       'DF Consolidado - Demonstração do Fluxo de Caixa (Método Indireto)',
       'DF Consolidado - Demonstração do Resultado',
       'DF Individual - Balanço Patrimonial Ativo',
       'DF Individual - Balanço Patrimonial Passivo',
       'DF Individual - Demonstração das Mutações do Patrimônio Líquido',
       'DF Individual - Demonstração de Resultado Abrangente',
       'DF Individual - Demonstração de Valor Adicionado',
       'DF Individual - Demonstração do Fluxo de Caixa (Método Indireto)',
       'DF Individual - Demonstração do Resultado'], dtype=object)